# Relationship between a Vancouver Neighbourhood and the Characteristics of its Trees

by Evan Ma

## Introduction

In many Vancouver residential neighbourhoods there are a lot of trees which contribute to the overall ambiance of the neighbourhood. Traditionally, a high density of trees in a neighbourhood is desirable to one with only a sparse density. In this study, we shall try to identify the genera of trees that are common to the neighbourhoods of Vancouver and whether there is any pattern of characteristics of trees found in Vancouver's neighbourhoods.

First, let's load the required packages.

In [ ]:
# Import required libraries
import pandas as pd
import altair as alt
alt.data_transformers.enable('default', max_rows=1000000)
import json
import numpy as np
from itertools import accumulate

We shall now load the provided 'vancouver_trees.csv' dataset and store it in the dataframe "trees_df". Let's also display a few records and obtain basic statistics.

In [ ]:
trees_url='https://raw.githubusercontent.com/UBC-MDS/exploratory-data-viz/main/data/vancouver_trees.csv'

trees_df=pd.read_csv(trees_url, parse_dates=['date_planted'])
#trees_df=pd.read_csv(trees_url)

trees_df.head()

Each record is composed of information about the location, characteristics (such as species, genus, location, etc) of a tree.

In [ ]:
trees_df.describe()

Checking the above statistics, we need to filter out those with a diameter under 1 inch.

In [ ]:
trees_df=trees_df[trees_df['diameter']>1]

trees_df.info()

In [ ]:
trees_df.describe()


The original dataset has 30000 records, so it is clear that 26 records have been filtered out. Let's take a look at the number of genera of trees in this study:

In [ ]:
len(trees_df['genus_name'].unique())

As there are a total of 82 genera, it will be prudent to limit our visualizations to the overall top 75% so that our maps are not too cluttered.

In [ ]:
genera_sort_list = trees_df.groupby(by='genus_name').count().sort_values(by='tree_id', ascending=False).reset_index()['genus_name'].to_list()
upper_proportion=.75
genera_percentage = (trees_df.groupby(by='genus_name').count().sort_values(by='tree_id', ascending=False).reset_index()['tree_id'] / len(trees_df)).to_list()

genera_percentage_df = pd.DataFrame({
  'Genus': genera_sort_list,
  'Portion': genera_percentage,
  'cdf': accumulate(genera_percentage)
})

upper_genera_list = genera_sort_list[0:sum(genera_percentage_df['cdf']<=upper_proportion)]
upper_genera_list

There are therefore nine genera that make up the top 75% of trees. We shall now construct a heat map to visualize the distribution of trees across the neighbourhoods. First, we shall rank the neighbourhoods and genera by the count of tree records.

In [ ]:
nbd_sort_list = trees_df.groupby(by='neighbourhood_name').count().sort_values(by='tree_id', ascending=True).reset_index()['neighbourhood_name'].to_list()

Also, a line plot of proportion of total number of trees by genera will be constructed

In [ ]:

genera_pc_plot = alt.Chart(genera_percentage_df).mark_line().encode(
    alt.X('Genus:N', title='Genus Name', sort=genera_sort_list, axis=alt.Axis(grid=True)),
    alt.Y('Portion:Q', scale=alt.Scale(zero=False), title='Portion of Total'),
    tooltip=['Genus', 'Portion']).properties(width=600, height=80)



A similar plot of proportion of total number of trees by neighbourhood is also contructed.

In [ ]:
nbd_percentage = (trees_df.groupby(by='neighbourhood_name').count().sort_values(by='tree_id', ascending=True).reset_index()['tree_id'] / len(trees_df)).to_list()

nbd_percentage_df = pd.DataFrame({
  'Neighbourhood': nbd_sort_list,
  'Portion': nbd_percentage,
  'cdf': accumulate(nbd_percentage)
})

nbd_pc_plot = alt.Chart(nbd_percentage_df).mark_bar(color='coral').encode(
    alt.Y('Neighbourhood:N', title='', sort=nbd_sort_list, axis=alt.Axis(grid=True, orient='right')),
    alt.X('Portion:Q', title='Portion of Total'),
    tooltip=['Neighbourhood', 'Portion']).properties(width=80, height=400)



In [ ]:
trees_counts_plot=alt.Chart(trees_df).mark_rect().encode(
    alt.Y('neighbourhood_name:N', 
          sort=nbd_sort_list,
          title='Neighbourhood Name'),
    alt.X('genus_name:N', sort=genera_sort_list, title='',
           axis=alt.Axis(labels=False)),
    alt.Color('count()', scale=alt.Scale(scheme='purplebluegreen'), title='Count'),
    tooltip=['neighbourhood_name', 'genus_name', 'count()']
    ).properties(width=600, height=400)

((trees_counts_plot & genera_pc_plot) | nbd_pc_plot).properties(
    title=alt.TitleParams('Heat Map for Visualization of Distribution of Trees by Genus and Neighbourhood',
                        subtitle='The top 3 Nieghbourhoods with the most trees are Kensington-Cedar Cottage, Renfrew-Collingwood, and Hastings-Sunrise')).configure_title(anchor='middle') 

It is clear that the most prevalent genera are Acer (Maple) amd Prunus (Plum), whereas the neighbourhoods with the most number of trees is Kensington-Cedar. Tree counts will also be visualized on a map of Vancouver.

In [ ]:
url_geojson = 'https://raw.githubusercontent.com/UBC-MDS/exploratory-data-viz/main/data/local-area-boundary.geojson'

data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))


In [ ]:
data_geojson_remote

A base map is first defined as the background against which other maps with trees' statistics can be overlaid.

In [ ]:
base_map = alt.Chart(data_geojson_remote).mark_geoshape(
    color = 'gray', opacity= 0.4, stroke='black').encode(
).project(type='identity', reflectY=True).properties(width=500, height=300)



Each tree will be marked on the map with color chanel assigned to the genus of each tree. Let's take a look at the distribution of the genera of trees that make up 75% of the total to limit the number of genera displayed, with the genera of trees displayed in the legend in descending order of count of records.

In [ ]:
select_genus = alt.selection_single(
    fields=['genus_name'],
    bind='legend')


all_trees_plot = alt.Chart(trees_df[trees_df['genus_name'].isin(upper_genera_list)]).mark_circle(size=10).encode(
    longitude='longitude',
    latitude='latitude',
    color=alt.Color('genus_name:N', title='Genus', sort=genera_sort_list[0:sum(genera_percentage_df['cdf']<=upper_proportion)]),
    tooltip=['neighbourhood_name']).project(type= 'identity', reflectY=True)

(base_map + all_trees_plot.encode(opacity=
                                  alt.condition(select_genus, alt.value(1.0), alt.value(0.05))).add_selection(select_genus)).configure_view(
    stroke='black').properties(title=alt.TitleParams('Map of Vancouver Overlaid with Trees of the top 75% of Genera',
                                              subtitle='Select Genus by clicking on its name on Legend'), height=600, width=800)

From the above map, it is evident that trees of genera other than Acer and Prunus are very sparse in their distributions across all neighbourhoods of Vancouver.

## Diameter and Height Range of Trees

Let's represent the diameters of trees in a series of boxplots by the neighbourhoods.

In [ ]:
dia_boxplots = alt.Chart(trees_df).mark_boxplot().encode(
    alt.X('diameter', type='quantitative', title='Diameter of Tree (inches)', scale=alt.Scale(zero=False)),
    alt.Y('neighbourhood_name', type='nominal', title='Neighbourhood Name'),
    tooltip=['tree_id']).properties(height=400, width=500, title=alt.TitleParams(
    'Boxplots of Diameters of Trees by Neighbourhood'))

dia_boxplots

There are five outliers where the diameter is bigger than 80 inches. Hence, let's exclude them when it is re-plotted.

In [ ]:
dia_boxplots = alt.Chart(trees_df).mark_boxplot().encode(
    alt.X('diameter', type='quantitative', title='Diameter of Tree (inches)', scale=alt.Scale(domain=(0, 80), zero=False)),
    alt.Y('neighbourhood_name', type='nominal', 
          sort=trees_df.groupby(by='neighbourhood_name').median().sort_values(by='diameter').reset_index()['neighbourhood_name'].to_list(), 
          title='Neighbourhood Name'), 
    tooltip=['tree_id']).properties(height=400, width=500, title=alt.TitleParams(
    'Boxplots of Diameters of Trees by Neighbourhood', subtitle='Medians range from 7 inches in Downtown to 13 inches in Kitsilano'))

dia_boxplots

It is interesting to see that the median of trees' diameters vary with neighbourhoods. Let's see if we can obtain more insight by visualizing both diameter and height range on a map.

To enable us to visualize height range, we first obtain average latitude and longitude coordinates of trees by region.

In [ ]:
mean_coord = trees_df.groupby(by='neighbourhood_name').mean().loc[:,'latitude':'longitude'].reset_index()

In [ ]:
trees_medians = trees_df.groupby(by='neighbourhood_name').median().loc[:, ['diameter', 'height_range_id']].reset_index()

mean_coord=trees_medians.merge(mean_coord, how='inner', left_on='neighbourhood_name', right_on='neighbourhood_name')


In [ ]:

mean_dia_map = alt.Chart(data_geojson_remote).mark_geoshape(opacity=1).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(mean_coord, 'neighbourhood_name', ['diameter', 'neighbourhood_name'])).encode(
    color=alt.Color('diameter:Q', title='Median Diameter', scale=alt.Scale(scheme='lightgreyteal')),
    tooltip=[alt.Tooltip('neighbourhood_name:N', title='Neighbourhood'),
             alt.Tooltip('diameter:Q', title='Median Diameter')]).project(type='identity', reflectY=True)

height_pts = alt.Chart(mean_coord).mark_circle().encode(
    longitude='longitude',
    latitude='latitude',
    size=alt.Size('height_range_id:Q', title='Median Height Range', scale=alt.Scale(scheme='set1', domain=(0,3))),
    tooltip=alt.Tooltip('height_range_id', title='Median Height Range')).project(type= 'identity', reflectY=True)

(mean_dia_map + height_pts).properties(height=400, width=600, title=
                                      alt.TitleParams('Map of Median Diameters and Height Range', 
                                          subtitle='Median Sizes of trees in Western Neighbourhoods are taller'))

From the above chart, we can see that bigger trees (median height around 30 ft and median diameter of 12 inches or more) tend to grow in western neighbourhoods such as West Point Grey, Dubar-Southlands, Kitsilano and Shaughnessy. As such, we are interested in a scatter plot of all trees by neighbourhood.

In [ ]:
genera_bar = genera_pc_plot.mark_bar()

select_interval = alt.selection_interval(encodings=['x'])

In [ ]:
bar_slider = genera_bar.add_selection(select_interval).encode(
    color=alt.condition(select_interval, alt.value('navy'), alt.value('lightgray'))).properties(height=150, width=1000)

select_neighbour = alt.selection_single(
    fields=['neighbourhood_name'],
    bind='legend')

stats_plot = alt.Chart(trees_df[trees_df['genus_name'].isin(upper_genera_list)]).mark_point().encode(
                alt.X('height_range_id:Q', scale=alt.Scale(nice=False), title='Height Range ID'),
                alt.Y('diameter:Q', scale=alt.Scale(type='log', nice=False), title='Diameter (inches)'),
                tooltip=['height_range_id:Q', 'diameter:Q', 'genus_name:N']).properties(
    title=alt.TitleParams('Diameter vs Height Range by Genus', subtitle='Select the Neighbourhood on the Legend'), 
                          width=500, height=500)

stats_plot.encode(opacity=alt.condition(select_neighbour, alt.value(0.8), alt.value(0)), color=alt.Color('neighbourhood_name:N', title='Neighbourhood Name')).add_selection(select_neighbour)


From the above we can see that diameter is positively correlated with the height of a tree, we would like to check the density plots of diameters of trees by genus amd neighbourhoods. Only the genera that makes up the top 75% of trees will be included.

In [ ]:
select_genera = alt.selection_single(
    fields=['genus_name'],
    bind='legend')

diameter_density_genus = alt.Chart(trees_df[trees_df['genus_name'].isin(
    upper_genera_list) & (trees_df['diameter'] < 65)]).transform_density(
    'diameter',
    groupby=['genus_name'],
    as_=['diameter', 'density'], steps=100).mark_area(opacity=0.5).encode(
    x=alt.X('diameter:Q'),
    y='density:Q',
    tooltip=alt.Tooltip('diameter:Q', title='Diameter'),
    color=alt.Color('genus_name', sort=upper_genera_list)).properties(width=120, height=120,
                                              title='')

diameter_density_genus.encode(opacity=alt.condition(select_genera, alt.value(0.8), alt.value(0.2))).add_selection(
    select_genera).facet('genus_name', columns=3).properties(
    title='Density Plots of Diameter of Trees by Tree Genus').configure_title(anchor='middle').resolve_scale(
    x='independent', y='independent')

For the top 2 genera (Acer and Prunus) that accounts for about 45% of the total trees population, Acer has a peak in density at around 3.6 inches, whereas for Prunus there are two peaks - at around 3.5 inches and 12.9 inches. Actually, the genus whose density has the highest peak for the sample is Tilia (Lindens) at 13.8 inches.

For our final visualization of distribution of trees across the neighbourhoods, we shall create a dataframe of counts of trees that is grouped by neighbourhood and genus name.

In [ ]:
count_by_nd_genus = trees_df.groupby(by=['neighbourhood_name', 'genus_name']).count().reset_index()
count_by_nd_genus = count_by_nd_genus.iloc[:,0:3].rename(columns={'std_street':'number'}).sort_values(by=['neighbourhood_name', 'number'], ascending=False).reset_index(drop=True)

# Find the percentage of each genus in each neighbourhood

sum_by_nd=count_by_nd_genus.groupby(by='neighbourhood_name').sum().reset_index().rename(columns={'number':'total'})
count_by_nd_genus = count_by_nd_genus.merge(sum_by_nd, how='inner', left_on='neighbourhood_name', right_on='neighbourhood_name')
count_by_nd_genus['percent_total'] = count_by_nd_genus['number'] / count_by_nd_genus['total']
count_by_nd_genus

We shall now construct a bar chart of the overall top 75% genera by number with a dropdown list for selection of neighbourhood.

In [ ]:
nbd_list=sorted(count_by_nd_genus['neighbourhood_name'].unique())

dropdown = alt.binding_select(name='Neighbourhood ', options=nbd_list)

select_nbd = alt.selection_single(
    fields=['neighbourhood_name'], 
    bind=alt.binding_select(
        name='Neighbourhood ', options=nbd_list))

#Define bar chart
count_bar = alt.Chart(count_by_nd_genus).transform_filter(select_nbd).mark_bar(color='coral').encode(
    alt.Y('genus_name:N', title='Genus Name',
          sort=upper_genera_list, 
          scale=alt.Scale(domain=upper_genera_list)),
    alt.X('number:Q', title='Count of Trees'))

count_bar.add_selection(select_nbd).properties(title='Bar Chart of Count of Trees by Genera')

From the above bar chart, we can see that the genera Acer and Prunus are consistently prevalent for all neighbourhoods of Vancouver.

## Dashboard

We shall now construct a dashboard of plots for an overall visualization of statistics of trees' characteristics by neighbourhood.

In [ ]:
select_diameter = alt.selection_interval()

brush = alt.selection(type='interval')

dia_slider = (alt.Chart(trees_df)).mark_bar().encode(
    alt.X('diameter:Q', title='Diameter in inches', scale=alt.Scale(domain=[1, 70])),
    alt.Y('count()', scale=alt.Scale(type = 'log', domain=[1, 6000])),
    tooltip=alt.Tooltip(['diameter', 'count()'])).properties(
    title='Count of trees by Diameter for the Neighbourhood Selected', height=150)


In [ ]:
dia_trees_plot = (mean_dia_map.properties(width=300, height=300, title='Neighbourhood Map Overlaid with Trees by Genus') + all_trees_plot.transform_filter(brush)
                  .transform_filter(select_nbd)) & dia_slider.transform_filter(
    select_nbd).properties(width=300)


In [ ]:
(((stats_plot.encode(opacity=alt.condition(brush, alt.value(0.8), alt.value(0.2)), 
                   color=alt.Color('genus_name:N', title='Genus Name'))
 .transform_filter(select_nbd).add_selection(brush).properties(
     title='Diameter vs Height Range ID - Drag and Select', width=300, height=300) & count_bar.add_selection(select_nbd)
 .properties(width=300, height=140, title='Count of Trees by Genus'))  | (dia_trees_plot ))).properties(
    title=alt.TitleParams('Dashboard of Plots', 
    subtitle='Select Neighbourhood by Dropdown Menu, Drag and Select Height Range and Diameter on Scatter Plot'))

## Discussion and Conclusion

There are 29974 records filtered out of a total 30000 in the original database. We think this is just a subset of the actual number of trees. Hence, the conclusions arrived here may not be truly representative as they will be subject to the method of sampling of the original dataset. 

Nevertheless, we have found that overall the following nine genera of trees account for the top 75% of trees in the dataset: Acer (Maple), Prunius (Plum), Fraxinus (Ash), Tilia, Quercus, Carpinus, Fagus, Malus, and Crataegus. In most neighbourhoods, the top 2 genera are Acer and Prunius, except for Downtown where the second place is not Prunius. However, this is suspect as there is an abundance of plum trees in downtown.


The median diameter of trees vary from 7 to 13 inches, with the ones having the largest diameters in the western neighbourhoods such as Kerrisdale (11 inches), Dunbar-Southlands (12 inches), Kitsilano (13 inches), West Point Grey (12 inches), and Shaunessey (12.5 inches). The median height of the trees in these regions are also in the 20's and 30's in feet. 

As tall trees contribute to the ambiance of a residential neighbourhood, an interesting extension to this project will be to investigate the assessment values of detached homes in these neighbourhoods, and check if the density of and proximity to tall trees are correlated with the per-square-feet value of these properties. Such data is also accessible from BC Assessment.

## References

- [Data Source](https://raw.githubusercontent.com/UBC-MDS/exploratory-data-viz/main/data/vancouver_trees.csv)